In [ ]:
# markdown 

In [1]:
# import section 
import pandas as pd,numpy as np, os
import shelve
import collections
import operator

In [2]:
all_jokes_ids = set(range(100))
column_nanes =   ['jokes_rated']+ list(map(lambda x: x,all_jokes_ids ))
jokeID_dataset  = { }
similarityDict ={}
userID_dataset ={}
for num in range(100):
    jokeID_dataset[num] = {}
    jokeID_dataset[num]['users_rated'] = { }
    jokeID_dataset[num]['mean_rating'] = None
    jokeID_dataset[num]['std_rating'] = None

In [3]:
#get data sets 
data_frame_master = pd.read_excel(os.getcwd()+'\\jester-data-1.xls', names= column_nanes )
data_frame_2 = pd.read_excel(os.getcwd()+'\\jester-data-2.xls', names= column_nanes )
data_frame_master = data_frame_master.append(data_frame_2, ignore_index=True)
data_frame_3 = pd.read_excel(os.getcwd()+'\\jester-data-3.xls', names= column_nanes )
data_frame_master =  data_frame_master.append(data_frame_3, ignore_index=True)
data_frame_master.index.names = ['user_id']
data_frame_master.reset_index(level=0, inplace=True)

In [4]:
data_frame_master.head()

,user_id,jokes_rated,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,1,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,2,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,3,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,4,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,...,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [4]:
data_frame_master.shape

(73418, 102)

In [25]:
# may need to clube the other data sets 

In [5]:
# sampel data set 
data_frame_sample =  data_frame_master.head(10000)

In [ ]:
###
###
# select full data set, if needed
# sampel data set 
data_frame_sample =  data_frame_master

In [6]:
data_frame_sample.head()

,user_id,jokes_rated,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,1,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,2,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,3,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,4,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,...,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [7]:
for index, row in data_frame_sample.iterrows():
    user_id = int(row.user_id)
    number_jokes_rated =  row.jokes_rated
    ratings = row[2:]
    ##########################
    userID_dataset[user_id] = {}
    userID_dataset[user_id]['jokes_rated'] ={}
    userID_dataset[user_id]['jokes_not_rated'] ={}
    userID_dataset[user_id]['mean_rating'] = None
    userID_dataset[user_id]['std_rating'] = None # standard deviation
    userID_dataset[user_id]['number_jokes_rated'] = number_jokes_rated
    ##########################
    jokes_ratings_by_user = row[2:].to_dict()
    jokes_rated_by_user = {k:v for (k,v) in jokes_ratings_by_user .items() if v < 99} # can yse this to get keys () which is the moves rated by a user
    jokes_not_rated_by_user = list(all_jokes_ids - jokes_rated_by_user.keys()) 
    list_of_ratings = list(jokes_rated_by_user.values())
    mean_rating_given_by_user = np.mean(list_of_ratings)
    std_rating_given_by_user =  np.std(list_of_ratings)
    for key in jokes_rated_by_user.keys():
        jokeID_dataset[key]['users_rated'][user_id] = jokes_rated_by_user[key]
    #########################    
    userID_dataset[user_id]['jokes_rated'] = jokes_rated_by_user
    userID_dataset[user_id]['jokes_not_rated'] = jokes_not_rated_by_user
    userID_dataset[user_id]['mean_rating'] = mean_rating_given_by_user
    userID_dataset[user_id]['std_rating'] = std_rating_given_by_user
    userID_dataset[user_id]['number_jokes_rated'] = number_jokes_rated

In [8]:
#run this after all the rows are read # can optiona. but on large ste of jokes can useful
for joke_id in jokeID_dataset.keys():
    jokeID_dataset[joke_id]['mean_rating'] = np.mean(list(jokeID_dataset[0]['users_rated'].values()))
    jokeID_dataset[joke_id]['std_rating'] = np.std(list(jokeID_dataset[0]['users_rated'].values()))

In [ ]:
# dump to disk 

In [3]:
disk = shelve.open('dump_to_disk')

In [4]:
# run this only when you want to load the data from disk
###
##
jokeID_dataset = disk['jokeID_dataset'] 
userID_dataset = disk['userID_dataset']
similarityDict = disk['similarityDict']

In [10]:
# do not run this, unless you want to over write the exisiting data. 
disk['jokeID_dataset'] = jokeID_dataset
disk['userID_dataset'] = userID_dataset
disk['similarityDict'] = similarityDict 

In [11]:
disk.close() 

In [5]:
def getRecordUser(userId):
    return userID_dataset[userId]

In [6]:
# we dont want info about all users, just those who rated the joke similar to similar to target user 
def usersWhoGaveSameRating(joke_id,diviationInterval,jokeWithMaximumRatingByUser,ratingOfTheJokeRatedMax):
    jokeRecord =jokeID_dataset[jokeWithMaximumRatingByUser]
    usersWhoRatedJoke = jokeRecord['users_rated']
    listOfUsers = []
    while len(listOfUsers) < 1:
        usersWhoRatedJoke = {k:v for (k,v) in usersWhoRatedJoke .items() if v < ratingOfTheJokeRatedMax + diviationInterval }
        usersWhoRatedJoke = {k:v for (k,v) in usersWhoRatedJoke .items() if v > ratingOfTheJokeRatedMax - diviationInterval }
        diviationInterval += 0.25
        listOfUsers = list(usersWhoRatedJoke.keys())
        usersWhoRatedJoke = jokeRecord['users_rated'] # reseting 
    return listOfUsers

In [7]:
# refered from 
#https://stackoverflow.com/questions/28140530/type-error-when-comparing-two-dictionaries-using-cosine-similarity-in-python

def calculateCosineSimilarity(v1, v2):
    up = 0
    for key in set(v1).intersection(v2):
        v1_value = v1[key]
        v2_value = v2[key]
        up += v1_value * v2_value
    if up == 0:
        return 0
    return up / (np.sqrt(np.dot(list(v1.values()), list(v1.values()))) * np.sqrt(np.dot(list(v2.values()), list(v2.values()))))

In [8]:

def getCosineSimilarity(userId,usersWhoRatedJoke,jokesRatedByUser):
    cosineDict = {}
    cosineDict[userId] = {}
    for targetId in usersWhoRatedJoke:
        targetRecord = getRecordUser(targetId)
        jokesRatedByTarget = targetRecord['jokes_rated']
        try:
            cosineDict[targetId] = similarityDict[userId][targetId]
        except KeyError:
            similarity = calculateCosineSimilarity(jokesRatedByTarget,jokesRatedByUser) 
            cosineDict[targetId] = similarity
            try:
                similarityDict[userId][targetId] = similarity
            except KeyError:
                similarityDict[userId] = {}
                similarityDict[userId][targetId] = similarity
            try:    
                similarityDict[targetId][userId] = similarity
            except KeyError:
                similarityDict[targetId] = {}
                similarityDict[targetId][userId] = similarity
    del cosineDict[userId]        
    return cosineDict
    

In [9]:
def getRecommendationForUser(userId, diviationInterval):
    userRecord = getRecordUser(userId)
    jokesRatedByUser = userRecord['jokes_rated']
    jokeWithMaximumRatingByUser = max(jokesRatedByUser, key=jokesRatedByUser.get)
    ratingOfTheJokeRatedMax = userRecord['jokes_rated'][jokeWithMaximumRatingByUser]
    jokeRecord =jokeID_dataset[jokeWithMaximumRatingByUser ]
    usersWhoRatedJoke = usersWhoGaveSameRating(jokeWithMaximumRatingByUser, diviationInterval,jokeWithMaximumRatingByUser,ratingOfTheJokeRatedMax )
    cosineSimilarityBetweenUsers = getCosineSimilarity(userId,usersWhoRatedJoke,jokesRatedByUser)
    cosineSimilarityBetweenUsers = sorted(cosineSimilarityBetweenUsers.items(), key = lambda x: x[1], reverse= True)
    iterateUserId = 0
    try:
        while(True):
            targetUserId=cosineSimilarityBetweenUsers[iterateUserId][0] 
            jokesRatedByTarget= userID_dataset[targetUserId]['jokes_rated']
            jokesRatedByTarget = {k:v for (k,v) in jokesRatedByTarget .items() if v >= userID_dataset[targetUserId]['mean_rating']}
            jokesNotRatedByUser = userID_dataset[userId]['jokes_not_rated']
            jokesOfIntrest =  set(jokesRatedByTarget) -  set(jokesNotRatedByUser)
            jokesOfNotIntrest = set(jokesRatedByTarget) -  jokesOfIntrest
            for key in jokesOfNotIntrest:
                try:
                    del jokesRatedByTarget[key]
                except KeyError:
                    pass
            try:
                recommendedJoke = max(jokesRatedByTarget.items(), key=operator.itemgetter(1))[0]
                return recommendedJoke
            except ValueError:
                iterateUserId += 1
    except IndexError:
        diviationInterval += 0.1
        getRecommendationForUser(userID, diviationInterval)

In [12]:
def main():
    while True:
        print('enter userId you want Recommendation for')
        userId = int(input())
        print('enter diviationInterval; should be around 0.25')
        diviationInterval = float(input())
        if userID_dataset[userId]['number_jokes_rated'] == 100:
            print('User has seen all the available jokes')
        else:
            jokeID = getRecommendationForUser(userId, diviationInterval)
            print('Joke recommended for userId', userId, ' is ', jokeID,'\n\n')
        
            print('press 1 to exit, 2 to continue')
            userInput = int(input())
            if userInput == 1:
                break
            else:
                continue
        

In [13]:
if __name__ == '__main__':
    main()

enter userId you want Recommendation for
1
enter diviationInterval; should be around 0.25
0.25
Joke recommended for userId 1  is  27 


press 1 to exit, 2 to continue
2
enter userId you want Recommendation for
0
enter diviationInterval; should be around 0.25
0.25
User has seen all the available jokes
enter userId you want Recommendation for
6
enter diviationInterval; should be around 0.25
0.25
User has seen all the available jokes
enter userId you want Recommendation for
8
enter diviationInterval; should be around 0.25
0.25
Joke recommended for userId 8  is  27 


press 1 to exit, 2 to continue
1
